In [8]:
import pandas as pd
import subprocess
import glob, os
import requests
from scipy import stats



In [9]:
#change these three lines to change what species you are running on
#location of the results of our pipeline within /outputs
output_folder_name = "f50_hg38_e100_mm39"
#dfam clade number for the target (first species in output folder name)
target_clade = 9606
#dfam clade number for the query (second species in output folder name)
query_clade = 10090


In [10]:
#converts the coverage CSV file for a family into a pandas dataframe
def get_family_coverage_dataframe(family_name, output_folder_name):
    file_name = "../outputs/" +  output_folder_name + "/alignments/" + family_name + "/repeat_alignment_coverage.csv"
    section_col_names= ["bp", "mis", "indel", "un"]
    left_col_names = ["left"+string for string in section_col_names]
    repeat_col_names = ["repeat"+string for string in section_col_names]
    right_col_names = ["right"+string for string in section_col_names]
    columns = ["id"] + left_col_names + repeat_col_names + right_col_names
    df = pd.read_csv(file_name,names=columns)
    return df

In [11]:
#get the names of the families that had at least one copy map to query
family_beds = glob.glob("../outputs/" + output_folder_name + "/target_beds/*.bed")
families = sorted([pth.split("/")[-1].split(".")[0]for pth in family_beds])

In [12]:
#get the families we expect to see in query from Dfam
url = "https://dfam.org/api/families"
params = {
    # The summary format is metadata-only and does not include
    # full details such as the consensus sequence and citations
    "format": "summary",

    # Only retrieve the first 10 results in this query


    # Search in query
    "clade": str(query_clade),

    # Include families from ancestor and descendant taxa in the results
    "clade_relatives": "both",
}
query_response = requests.get(url, params)
results = query_response.json()["results"]
query_families = [family['name'] for family in (query_response.json()['results'])]
query_family_lengths = [family['length'] for family in (query_response.json()['results'])]



In [13]:
#get the families we expect to see in target from Dfam
url = "https://dfam.org/api/families"
params = {
    # The summary format is metadata-only and does not include
    # full details such as the consensus sequence and citations
    "format": "summary",

    # Only retrieve the first 10 results in this query


    # Search in target
    "clade": str(target_clade),

    # Include families from ancestor and descendant taxa in the results
    "clade_relatives": "both",
}
target_response = requests.get(url, params)
target_families = [family['name'] for family in (target_response.json()['results'])]
target_family_lengths = [family['length'] for family in (target_response.json()['results'])]


In [14]:
# match family names to Dfam's families (adding/subtracting suffixes where we need to to find a match)
# we discard families that we can't match to the dfam results for either target or query
family_name_dict = {}
shared_families_trimmed = []
unshared_families_trimmed = []
unidentified_families = []
for family in families:
    if family in query_families:
        family_name_dict[family] = family
        shared_families_trimmed.append(family)
    elif family in target_families:
        family_name_dict[family] = family
        unshared_families_trimmed.append(family)
    elif family[-4:] == "-int":
        trimmed_family = family[:-4]
        if trimmed_family in query_families:
            family_name_dict[family] = trimmed_family
            shared_families_trimmed.append(family)
        elif trimmed_family in target_families:
            family_name_dict[family] = trimmed_family
            unshared_families_trimmed.append(family)
        else:
            print(family)
    elif family+"_orf2" in query_families:
        family_name_dict[family] = family+"_orf2"
        shared_families_trimmed.append(family)
    elif family+"_3end" in query_families:
        family_name_dict[family] = family+"_3end"
        shared_families_trimmed.append(family)
    elif family+"_5end" in query_families:
        family_name_dict[family] = family+"_5end"
        shared_families_trimmed.append(family)
    elif family+"_orf2" in target_families:
        family_name_dict[family] = family+"_orf2"
        unshared_families_trimmed.append(family)
    elif family+"_3end" in target_families:
        family_name_dict[family] = family+"_3end"
        unshared_families_trimmed.append(family)
    elif family+"_5end" in target_families:
        family_name_dict[family] = family+"_5end"
        unshared_families_trimmed.append(family)
    else:
        unidentified_families.append(family)
print(unidentified_families)


['%GAATG%n', 'ALR%Alpha', 'Alu', 'BSR%Beta', 'Charlie7b_Mars', 'L1M', 'L1P', 'L1P5', 'Tigger1a_Art', 'Tigger1a_Mars']


In [15]:
#add manually ALR/Alpha and BSR/Beta which are named differently (ASRa and BSRb) by Dfam 3.7
unshared_unidentified_families = ['ALR%Alpha', 'BSR%Beta' ]
family_name_dict['ALR%Alpha'] = "ALRa"
family_name_dict['BSR%Beta'] = "BSRb"
unshared_families_trimmed = unshared_families_trimmed + unshared_unidentified_families


In [16]:
#create a dictionary of lengths of the families so we can use this in our thresholding
lengthdict = {}
for family in shared_families_trimmed:
    idx = query_families.index(family_name_dict[family])
    lengthdict[family] = query_family_lengths[idx]
for family in unshared_families_trimmed:
    idx = target_families.index(family_name_dict[family])
    lengthdict[family] = target_family_lengths[idx]
print(lengthdict)

{'5S': 121, '7SK': 331, '7SLRNA': 320, 'AmnHarb1': 1444, 'AmnL2-1': 2613, 'AmnSINE1': 575, 'AmnSINE2': 358, 'Arthur1': 3947, 'Arthur1A': 177, 'Arthur1B': 1225, 'Arthur1C': 363, 'Arthur2': 3700, 'BLACKJACK': 2969, 'CR1-11_Crp': 1461, 'CR1-12_AMi': 614, 'CR1-13_AMi': 724, 'CR1-16_AMi': 851, 'CR1-1_Amn': 813, 'CR1-3_Croc': 3605, 'CR1-L3A_Croc': 4288, 'CR1-L3B_Croc': 3277, 'CR1_Amni-1': 1319, 'CR1_Mam': 2204, 'Chap1_Mam': 342, 'Chap1a_Mam': 1038, 'Charlie1': 2781, 'Charlie10': 2822, 'Charlie10a': 280, 'Charlie10b': 242, 'Charlie11': 2196, 'Charlie12': 2870, 'Charlie13a': 1514, 'Charlie13b': 512, 'Charlie14a': 1166, 'Charlie15a': 224, 'Charlie15b': 1061, 'Charlie16': 3051, 'Charlie16a': 342, 'Charlie17': 2916, 'Charlie17a': 217, 'Charlie17b': 1204, 'Charlie18a': 342, 'Charlie19a': 386, 'Charlie1a': 1455, 'Charlie1b': 523, 'Charlie20a': 807, 'Charlie21a': 1213, 'Charlie22a': 491, 'Charlie23a': 339, 'Charlie24': 2449, 'Charlie25': 2524, 'Charlie26a': 325, 'Charlie29a': 748, 'Charlie29b': 1194

In [17]:
#create a dictionary of the coverage/csv dataframes for each family so we don't need to recalculate this multiple times for each family
coverage_dataframes = {}
for family in families:
    coverage_dataframes[family] = get_family_coverage_dataframe(family, output_folder_name)

In [26]:
#Set the thresholds for the alignment
#max_repeat_nonmatch is the maximum percentage of basepairs of the target we allow to be unmatched (deleted, mismatched or excluded from the alignment) within the repeat
max_repeat_nonmatch = 30.0
#max_side_nonmatch is the maximum percentage of basepairs of the target we allow to be unmatched on both sides of the repeat (ie at least one side must have less than this threshold unmatched)
max_side_nonmatch = 50.0
#length cap is the length at or above which we consider an alignment to meet the length threshold regardless of the expected length
length_cap = 300
#fraction of full length is the fraction of the whole length of the repeat that the repeat identified by repeatmasker that the repeat must span
fraction_of_full_length = 0.5
#create a dictionary that stores the number of repeat instances meeting these thresholds for each family
num_meeting_requirement = {}
for family in shared_families_trimmed + unshared_families_trimmed:
    family_coverage = coverage_dataframes[family]

    repeat_threshold = (family_coverage["repeatindel"] + family_coverage["repeatun"] + family_coverage["repeatmis"] <= max_repeat_nonmatch)
    length_threshold = (family_coverage["repeatbp"]>= min(length_cap,fraction_of_full_length*lengthdict[family_name]) )
    left_threshold = (family_coverage["leftindel"] + family_coverage["leftun"] + family_coverage["leftmis"] <= max_side_nonmatch)
    right_threshold = (family_coverage["rightindel"] + family_coverage["rightun"] + family_coverage["rightmis"] <= max_side_nonmatch)
    qualifying_rows = family_coverage.loc[ repeat_threshold & length_threshold &  (left_threshold|right_threshold)]
    num_meeting_requirement[family] = len(qualifying_rows.index)


In [27]:
#find out how many meet the requirement among the shared and the unshared repeats
shared_meeting_requirement = []
unshared_meeting_requirement = []
for family in shared_families_trimmed:
    shared_meeting_requirement.append((family,num_meeting_requirement[family]))
for family in unshared_families_trimmed:
    unshared_meeting_requirement.append((family,num_meeting_requirement[family]))


In [28]:
#threshold is the number of alignments that meet the requirements, at or above which we count an element as shared
threshold = 1

#positive = we say it's shared based on the alignments
#negative = we say it's not shared based on the alignments
false_positives = [item for item in unshared_meeting_requirement if item[1]>=threshold]
true_positives = [item for item in shared_meeting_requirement if item[1]>=threshold]

false_negatives = [item for item in shared_meeting_requirement if item[1]<threshold]
true_negatives = [item for item in unshared_meeting_requirement if item[1]<threshold]

print("FN:", len(false_negatives))
print("TN:", len(true_negatives))
print("FP:", len(false_positives))
print("TP:", len(true_positives))
print("Errors: False positives (we expect these families NOT to be shared but we are seeing enough alignments meeting the thresholds)")
print(false_positives)
print("Errors: False negatives (we expect these families  to be shared but we are NOT seeing enough alignments meeting the thresholds)")
print(false_negatives)

for false_positive in false_positives:
    print(false_positive[0], end=",")


FN: 188
TN: 466
FP: 29
TP: 586
Errors: False positives (we expect these families NOT to be shared but we are seeing enough alignments meeting the thresholds)
[('AluJr', 1), ('AluSx', 1), ('COMP-subunit_TAF11_rnd-6_family-27360', 2), ('COMP-subunit_TELO_rnd-6_family-166', 2), ('COMP-subunit_VNTR_rnd-6_family-8746', 1), ('DNM1r', 22), ('ERV24B_Prim-int', 1), ('FRAM', 1), ('HERV1_LTRa', 1), ('HERVIP10FH-int', 1), ('HSAT5v1', 12), ('HSAT5v2', 3), ('L1M2', 1), ('L1P1', 3), ('L1P3', 2), ('L1P4', 4), ('L1PA15', 1), ('L1PA8A', 1), ('L1PB', 1), ('L1PREC2', 1), ('LTR29', 1), ('LTR3B', 1), ('LTR44', 1), ('LTR71B', 1), ('MER4-int', 4), ('MSR1', 10), ('PRIMA4-int', 1), ('SVA_A', 1), ('Tigger3', 1)]
Errors: False negatives (we expect these families  to be shared but we are NOT seeing enough alignments meeting the thresholds)
[('5S', 0), ('Charlie10a', 0), ('Charlie10b', 0), ('Charlie12', 0), ('Charlie17b', 0), ('Charlie6', 0), ('Cheshire', 0), ('ERV3-16A3_I', 0), ('ERVL-B4-int', 0), ('ERVL-int', 0),

In [29]:
#code that fetches the csv rows of the alignments that are qualifying it according to our thresholds (useful for looking at false positives)
#can change family_name to look at different families
family_name = "MLT1A0"
family_coverage = coverage_dataframes[family_name]
repeat_threshold = (family_coverage["repeatindel"] + family_coverage["repeatun"] + family_coverage["repeatmis"] <= max_repeat_nonmatch)
length_threshold = (family_coverage["repeatbp"]>=fraction_of_full_length*lengthdict[family])
left_threshold = (family_coverage["leftindel"] + family_coverage["leftun"] + family_coverage["leftmis"] <= max_side_nonmatch)
right_threshold = (family_coverage["rightindel"] + family_coverage["rightun"] + family_coverage["rightmis"] <= max_side_nonmatch)
qualifying_rows = family_coverage.loc[ repeat_threshold & length_threshold &  (left_threshold|right_threshold)]
num_meeting_requirement[family] = len(qualifying_rows.index)

qualifying_rows

,id,leftbp,leftmis,leftindel,leftun,repeatbp,repeatmis,repeatindel,repeatun,rightbp,rightmis,rightindel,rightun
531,MLT1A0;LTR/ERVL-MaLR;880408;1062119,100,20.0,0.0,17.0,169,13.02,1.78,0.0,100,23.0,7.0,15.0
2638,MLT1A0;LTR/ERVL-MaLR;4588631;5535167,100,20.0,5.0,2.0,346,20.81,6.07,0.0,100,25.0,1.0,2.0


In [22]:
#calculate the number of alignments we did/repeats that make it through our filters for each repeat
num_csv_entries_dict = {}
for family_name in families:
    file_name = "../outputs/" +  output_folder_name + "/alignments/" + family_name + "/repeat_alignment_coverage.csv"
    with open(file_name, 'r') as fp:
        lines = len(fp.readlines())
        num_csv_entries_dict[family_name] = lines

In [23]:
#print out our false negative families and their abundance after filtering/number of alignments
for (fn, num_align_qualifying) in false_negatives:
    print(fn,num_csv_entries_dict[fn])

5S 180
Arthur1A 245
BLACKJACK 220
CR1-L3A_Croc 27
CR1-L3B_Croc 9
Charlie1 188
Charlie10a 45
Charlie10b 47
Charlie12 2
Charlie17b 100
Charlie1a 624
Charlie4z 1173
Charlie6 18
Charlie9 168
Cheshire 85
DIRS-1a_Amnio 471
DIRS-1b_Amnio 511
DIRS-1c_Amnio 506
ERV3-16A3_I 455
ERVL-B4-int 117
ERVL-int 45
ERVL40-int 81
EUTREP6 1
Eulor12 31
Eulor2B 27
Eulor2C 16
Eulor5B 18
Eulor6C 9
Eulor7 4
EutTc1-N1 6
EutTc1-N2 28
EuthAT-2 121
EuthAT-N1a 15
Eutr2B 80
FordPrefect 76
FordPrefect_a 30
HERVL-int 47
HERVL74-int 13
HY1 69
HY3 79
HY4 24
HY5 6
Kanga1 107
Kanga1b 53
L1M3 644
L1M3a 23
L1M3b 37
L1M3c 70
L1M3d 42
L1M3de 23
L1M3e 36
L1M4a1 235
L1M4a2 131
L1M4b 217
L1M4c 158
L1M6B 118
L1MA7 986
L1MA8 1213
L1MB1 590
L1MB2 851
L1MB3 1922
L1MB4 895
L1MC1 1199
L1MC2 572
L1MCb 225
L1MCc 322
L1MD1 505
L1MD2 1116
L1MDa 494
L1MDb 65
L1ME2z 855
L1ME3 1058
L1ME3B 968
L1MEa 23
L1MEc 1457
L1MEg1 74
L1MEg2 79
L1MEj 361
L2-1_Crp 26
LTR06 14
LTR102_Mam 88
LTR103_Mam 151
LTR105_Mam 81
LTR107_Mam 72
LTR108b_Mam 1
LTR108d_Mam

In [24]:
#t-test: are we more likely to detect a family to be shared if it is shorter -- yes
tp_len = [lengthdict[family] for (family, _) in true_positives]
fn_len =  [lengthdict[family] for (family, _) in false_negatives]
print("mean length of TP:", sum(tp_len)/len(tp_len))
print("mean length of FN:",sum(fn_len)/len(fn_len))
print("t-test:", stats.ttest_ind(tp_len, fn_len))


mean length of TP: 1020.2850467289719
mean length of FN: 908.5202312138729
t-test: TtestResult(statistic=1.3001215647355127, pvalue=0.19394732824629082, df=772.0)


In [25]:
#t-test: are we more likely to detect a family to be shared if it is more numerous -- yes
tp_num = [num_csv_entries_dict[family] for (family, _) in true_positives]
fn_num =  [num_csv_entries_dict[family] for (family, _) in false_negatives]
print("mean number of repeats of TP:",sum(tp_num)/len(tp_num))
print("mean number of repeats of FN:",sum(fn_num)/len(fn_num))
print("t-test:", stats.ttest_ind(tp_num, fn_num))


mean number of repeats of TP: 1035.2359813084113
mean number of repeats of FN: 190.72543352601156
t-test: TtestResult(statistic=3.8884916096214472, pvalue=0.00010955179447281612, df=772.0)
